# Pretrained models and fine tuning

- PRETRAINED MODEL: We can use a pretrained CNN, at the star of our model. This allow us to have a very good convolutional base before adding our own dense layeres classifier at the end. By using this technique we can train a very good classifier for a realtively small dataset ( < 10000 images )
- FINE TUNING: We want to tweak the final layers in our convolutional base to work better for our specific problem. This involves not touching or retraining the earlier layers in the convolutional base but only adjusting the final few.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
keras = tf.keras

In [ ]:
import tensorflow_datasets as tfds

(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'cats_vs_dogs', #nombre del dataset que se quiere cargar
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'], #80%entrenamiento, 10%validacion, 10%prueba
    with_info=True, #info adicional del dataset en el entorno, como los nombres de las clases y estadisticas
    as_supervised=True, #carga los datos en formato de a pares en lugar de un formato no supervisado
    shuffle_files=True #baraja los archivos del dataset antes de cargarlos para mejorar la generalizacion del modelo
)

In [ ]:
get_label_name = metadata.features['label'].int2str #creates a function object that we can use to get labels

#display 2 images from the dataset
for image, label in raw_train.take(2):
    plt.figure()
    plt.imshow(image)
    plt.title(get_label_name(label))

# Data Preprocessing

Since the sizes of our images are all different, we need to convert them all to the same size. We create a function that will do that for us below.

In [ ]:
IMG_SIZE = 160

def format_example (image, label):
    """
    returns an image that it reshaped to IMG_SIZE
    """

    image = tf.cast(image, tf.float32)
    image = (image/127.5) - 1
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    return image, label

Now we can apply this function to all our images using map

In [ ]:
train = raw_train.map(format_example)
validation = raw_validation.map(format_example)
test = raw_test.map(format_example)

Looking at the images reshaped

In [ ]:
for image, label in train.take(2):
    plt.figure()
    plt.imshow(image)
    plt.title(get_label_name(label))

Finally, we will shuffle and batch the images:

In [ ]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)


Now if we look at the shape of an original image vs the new image we will see it has been changed

In [ ]:
for img,label in raw_train.take(2):
    print("original shape: ", img.shape)

for img, label in train.take(2):
    print ('new shape', img.shape)
    

# Picking a pretrained model

The model will be used as the concolutional base for our model is the MobileNet V2 developed at Google. This model is trained on 1.4 millions images and has 1000 different classes.

We want to use this model but only its convolutional base. So when we load in the model we'll specify that we don't want to load the top classification layer. We'll tell the model what input shape to expect and to use the predetermined weights from imagenet. (Google dataset)

In [ ]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

#Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2 (input_shape=IMG_SHAPE, #tamanio de las imagenes
                                                include_top = False, #se debe exluuir la capa superior del modelo original de MobileNetV2, que es la capa de clasificacion final
                                                weights = 'imagenet') #se deben cargar los pesos preentrenados del modelo que han sido entrenados previamente en el conjunto de datos ImageNet

In [ ]:
base_model.summary()

In [ ]:
for image, _ in train_batches.take(1): #train_batches es un datast de tensorflow que contiene lotes de imagenes y sus etiquetas, toma el primer lote del ds
    pass #tera sobre el primer lote del dataset. Aquí, image representa un lote de imágenes, y _ es una variable que representa las etiquetas asociadas, que no se utilizan en este fragmento de código.

feature_batch = base_model(image) #pasa el lote de imagenes a traves del modelo preentrenado MobileNetV2 (sin la capa superior)
print(feature_batch.shape) #Imprime la forma del tensor feature_batch. La forma típica de estas características es (batch_size, height, width, channels)


# Freezing the Base

The term freezing refers to disabling the training property of a layer. It simply means we wont make any changes to the weights of any layers that are frozen during training.
- This is important as we don't want to change the convolutional base that already has learned weights.


In [ ]:
base_model.trainable = False

# Adding our classifier 

Now that we have our base layer setup we can add the classifier. Instead of flattening the feature map of the base layer we will use a global average pooling layer that will average the entire 5x5 area of each 2D feature map and return to us a single 1280 element vector per filter

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

Finally we will add the prediction layer that will be a single dense neuron. We can do this because we only have two classes to predict for

In [ ]:
prediction_layer = keras.layers.Dense(1)

Now we will combine these layers together in a model

In [ ]:
model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    prediction_layer
]
)

# Training the model

Now we will train and compile the model. We will use a very small learning rate to ensure that the model does not have any major changes made to it.

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss= tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [ ]:
# We can evaluate the model right now to see how it does before training it on our new images

initial_epochs = 3
validation_steps = 20

loss0, accuracy0 = model.evaluate(validation_batches, steps= validation_steps)


In [ ]:
#now we can train it on our images

history = model.fit(train_batches,
                    epochs=initial_epochs,
                    validation_data = validation_batches)

acc = history.history['accuracy']
print(acc)

We can save the model in keras like this:

In [ ]:
model.save('dogs_vs_cats.h5')
new_model = tf.keras.models.load_model('dogs_vs_cats.h5')